In [46]:
!nvidia-smi

Tue May 18 15:53:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    29W /  70W |    350MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%pip install -q tensorflow-addons[tensorflow]
# %pip install -q simpleitk
# %pip install -q colab_ssh
# %pip install -q wandb

     |████████████████████████████████| 686kB 28.0MB/s 


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_addons as tfa
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image
import glob
from tqdm.notebook import tqdm
from time import time
%matplotlib inline

In [3]:
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla T4, compute capability 7.5
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale


In [4]:
%cd /content
!rm -rf 3D_res_attn_Unet

/content


In [5]:
!git clone https://github.com/ShivamShrirao/3D_res_attn_Unet
%cd 3D_res_attn_Unet

Cloning into '3D_res_attn_Unet'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 38 (delta 17), reused 32 (delta 11), pack-reused 0
Unpacking objects: 100% (38/38), done.
/content/3D_res_attn_Unet


In [6]:
from model.conv_blocks import ConvNorm, NormAct, down_stack, up_stack

In [7]:
BATCH_SIZE = 2

In [24]:
config_defaults = {
    'activation'            : 'leaky_relu',
    'batch_size'            : BATCH_SIZE,
    'clr_step_size'         : 6,
    'groups'                : 'max',
    'dp_rate'               : 0.2,
    'dropout_type'          : 'Spatial',
    'frac_dv'               : 0.5,
    'loss_function'         : 'FocalTversky',
    'max_lr'                : 8e-4,
    'min_lr'                : 1e-6,
    'nheads'                : 8,
    'norm'                  : 'bn',
    'optimizer'             : 'adamw',
    'self_attn'             : False,
    'vary_weight_decay'     : True,
    'weight_decay'          : 1e-4,     # max weight decay
}

In [25]:
CONFIG = config_defaults

In [26]:
depth,height,width = 128,192,160

In [27]:
def enc_dec(x, frac_dv, stack_args):     # x(64,64,96,80)
    x1 = down_stack(x ,  64, blocks=2, strides=2, **stack_args)     # ( 256,32,48,40)
    x2 = down_stack(x1, 128, blocks=3, strides=2, **stack_args)     # ( 512,16,24,20)
    x3 = down_stack(x2, 256, blocks=4, strides=2, frac_dv=frac_dv, **stack_args) # (1024, 8,12,10)

    x4 = down_stack(x3, 256, blocks=3, strides=2, frac_dv=frac_dv, **stack_args) # (1024, 4, 6, 5)

    y = up_stack(x4,x3, 128, blocks=3, strides=2, frac_dv=frac_dv, **stack_args) # (1024+1024->512,8,12,10)
    y = up_stack(y, x2,  64, blocks=4, strides=2, frac_dv=frac_dv, **stack_args) # (512+512->256, 16,24,20)
    y = up_stack(y, x1,  32, blocks=3, strides=2, **stack_args)     # ( 256+256->128,32,48,40)
    y = up_stack(y, x ,  16, blocks=2, strides=2, **stack_args)     # ( 128+ 64->64,64,96,80)
    return y


def build_network(cfg, input_shape=(4,128,192,160), classes=3):
    stack_args = {'activation': cfg['activation'], 'groups': cfg['groups'], 'norm': cfg['norm'],
                  'dp_rate': cfg['dp_rate'], 'dropout_type': cfg['dropout_type']}

    inp = layers.Input(shape=input_shape)                           # ( 4,128,192,160)
    x = inp
    x = ConvNorm(32, kernel_size=3, strides=2, activation=cfg['activation'], norm=cfg['norm'])(x)   # (32,64,96,80)
    x = ConvNorm(64, kernel_size=3, do_norm_act=False)(x)                               # (64, 64, 96, 80)

    x = enc_dec(x, cfg['frac_dv'], stack_args)                                          # (64, 64, 96, 80)

    x = NormAct(activation=cfg['activation'], norm=cfg['norm'])(x)
    x = layers.UpSampling3D(data_format="channels_first")(x)                            # (64,128,192,160)
    x = ConvNorm(16, kernel_size=3, activation=cfg['activation'], norm=cfg['norm'])(x)  # (16,128,192,160)
    x = ConvNorm(classes,kernel_size=3, activation=cfg['activation'], norm=cfg['norm'])(x)#(3,128,192,160)
    x = layers.Softmax(axis=1)(x)         # softmax cause each pixel has unique class, no overlap with other classes, verified.

    return tf.keras.Model(inputs=inp, outputs=x)

In [28]:
model = build_network(CONFIG, input_shape=(4,depth,height,width), classes=3)

In [29]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_51 (InputLayer)           [(None, 4, 128, 192, 0                                            
__________________________________________________________________________________________________
conv_norm_8 (ConvNorm)          (None, 32, 64, 96, 8 3584        input_51[0][0]                   
__________________________________________________________________________________________________
conv_norm_9 (ConvNorm)          (None, 64, 64, 96, 8 55360       conv_norm_8[0][0]                
__________________________________________________________________________________________________
attn_bottleneck_block_48 (AttnB (None, 256, 32, 48,  42064       conv_norm_9[0][0]                
____________________________________________________________________________________________

In [30]:
model_plot = tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, dpi=None)